In [5]:
import warnings
warnings.filterwarnings('ignore')

import time

import pandas as pd
df = pd.read_csv('ssbpts.csv')

df2 = df.query('MidLat > 30 and MidLat < 75 and MidLon < -60 and MidLon > -135 ')

In [6]:
try:
    dfe = pd.read_csv('errors.csv', index_col=0)
    df3 = pd.concat([df2, dfe], axis = 1)
    df4 = df3.query('Status != Status')
except FileNotFoundError:
    dfe = pd.DataFrame()
    df4 = df2
    


In [ ]:
from obspy import UTCDateTime

from obspy.taup import TauPyModel

from obspy.clients.fdsn import Client

client = Client("IRIS")

model = TauPyModel('ak135')



def save_error_dataframe(dataframe):
    dfe.to_csv('errors.csv')

for bp in df4.iterrows():
    etime = UTCDateTime(bp[1].EvtTime)
    sta = bp[1].Station
    net = bp[1].Network
    deldeg = bp[1].Range
    
    print("Working on data from %3s.%5s on %15s for bpt %10d" % (net,sta,etime,bp[0]))
    
    arrivals = model.get_travel_times(0.0, deldeg, phase_list = ['SS'])
    
    
    time_ss = arrivals[0].time
    
    t1 = etime + time_ss - 200
    t2 = etime + time_ss + 200
    
    clock=time.time()
    try:
        st = client.get_waveforms(net, sta, "*", "LH*", t1, t2, attach_response = True)
    except:
        err = 'No Data at %s.%s' % (net, sta)
        #print(err)
        dfe.loc[bp[0],"Status"] = err
        continue
    print('  step completed in %8.2e s' % (time.time() - clock) )
        
    

    try:
        st.remove_response(output='ACC', pre_filt=(0.005, 0.01, 0.5, 1.0))
    except ValueError as e:
        err = 'Response Error: %s' % e
        #print(err)
        dfe.loc[bp[0],"Status"] = err
    
    try:
        inv = client.get_stations(
            starttime=t1, endtime=t2, 
            network=net, sta=sta, 
            loc="*", channel="LH*", level="response")
        
        st.rotate('->ZNE', inventory = inv)
        st.rotate('NE->RT', back_azimuth = bp[1].BAzimuth)
    except ValueError as e:
        err = 'Rotation Error: %s' % e
        #print(err)
        dfe.loc[bp[0],"Status"] = err
        
        continue
    
    bpid = '%012d' % bp[0]
    
    filename = 'data/mseed/' + bpid + '.mseed'
    
    trT = st[0]
    
    for tr in st:
        if 'T' in tr.stats.channel:
            trT = tr
    
    trT.resample(1)
    
    trT.write(filename)
    
    dfe.loc[bp[0],"Status"] = 'OK'
    
    save_error_dataframe(dfe)
   